In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt

# Load the Excel data for West Bengal and Uttar Pradesh
wb_data = pd.read_csv('finalcompileddatawb.csv')
up_data = pd.read_csv('finalcompileddataup.csv')
bihar_data = pd.read_csv('finalcompileddatabihar.csv')

# Strip and clean column names first
wb_data.columns = wb_data.columns.str.strip()
up_data.columns = up_data.columns.str.strip()
bihar_data.columns = bihar_data.columns.str.strip()

# Convert 'Year' safely
wb_data = wb_data[~wb_data['Year'].isna()]  # Drop rows where Year is NaN
up_data = up_data[~up_data['Year'].isna()]  # Drop rows where Year is NaN
bihar_data = bihar_data[~bihar_data['Year'].isna()]  # Drop rows where Year is NaN

wb_data['Year'] = wb_data['Year'].astype(str).str[:4].astype(int)
up_data['Year'] = up_data['Year'].astype(str).str[:4].astype(int)
bihar_data['Year'] = bihar_data['Year'].astype(str).str[:4].astype(int)

# Define variables for both regions
regions = {
    'West Bengal': {
        'data': wb_data,
        'years': wb_data['Year'],
        'rainfall': wb_data['Rain Fall (in millimeters)'],
        'temperature': wb_data['Temperature'],
        'npk': wb_data['Amount of NPK Used'],
        'irrigated_area': wb_data['Net irrigated Area (in Hectare)'],
        'yield': wb_data['Yield of Wheat (kg/Hectare)']
    },
    'Uttar Pradesh': {
        'data': up_data,
        'years': up_data['Year'],
        'rainfall': up_data['Rain Fall (in millimeters)'],
        'temperature': up_data['Temperature'],
        'npk': up_data['Amount of NPK Used'],
        'irrigated_area': up_data['Net irrigated Area (in Hectare)'],
        'yield': up_data['Yield of Wheat (kg/Hectare)']
    },
    'Bihar': {
        'data': bihar_data,
        'years': bihar_data['Year'],
        'rainfall': bihar_data['Rain Fall (in millimeters)'],
        'temperature': bihar_data['Temperature'],
        'npk': bihar_data['Amount of NPK Used'],
        'irrigated_area': bihar_data['Net irrigated Area (in Hectare)'],
        'yield': bihar_data['Yield of Wheat (kg/Hectare)']
    }
}

# Section 4.1: Wheat Yield and Time Trend
def calculate_time_trend_correlation(region_data, region_name):
    correlation, _ = pearsonr(region_data['years'], region_data['yield'])
    print(f"{region_name} - Correlation between wheat yield and time trend: {correlation:.2f}")
    return correlation

# Section 4.2 and 4.3: Time Series Regression and OLS Regression
def perform_ols_regression(region_data, region_name):
    X_time = sm.add_constant(region_data['years'] - region_data['years'].min())
    y = region_data['yield']
    ols_model = sm.OLS(y, X_time).fit()
    print(f"\n{region_name} - OLS Regression Summary:")
    print(ols_model.summary())
    return ols_model

# Section 4.4: Robust Regression
def perform_robust_regression(region_data, region_name, ols_model):
    X_time = sm.add_constant(region_data['years'] - region_data['years'].min())
    y = region_data['yield']
    
    # Huber robust regression
    huber_model = sm.RLM(y, X_time, M=sm.robust.norms.HuberT()).fit()
    # Bisquare robust regression
    bisquare_model = sm.RLM(y, X_time, M=sm.robust.norms.TukeyBiweight()).fit()
    
    # Calculate forecasting errors
    ols_pred = ols_model.predict(X_time)
    huber_pred = huber_model.predict(X_time)
    bisquare_pred = bisquare_model.predict(X_time)
    
    errors = {
        'OLS': {
            'MAE': mean_absolute_error(y, ols_pred),
            'MAPE': mean_absolute_percentage_error(y, ols_pred)
        },
        'Huber': {
            'MAE': mean_absolute_error(y, huber_pred),
            'MAPE': mean_absolute_percentage_error(y, huber_pred)
        },
        'Bisquare': {
            'MAE': mean_absolute_error(y, bisquare_pred),
            'MAPE': mean_absolute_percentage_error(y, bisquare_pred)
        }
    }
    
    print(f"\n{region_name} - Forecasting Errors:")
    for model, metrics in errors.items():
        print(f"{model} - MAE: {metrics['MAE']:.2f}, MAPE: {metrics['MAPE']:.2%}")
    
    return ols_model, huber_model, bisquare_model

# Section 4.5: Adjusted Wheat Yield
def adjust_wheat_yield(region_data, region_name, model):
    base_year = 2023
    base_year_idx = region_data['years'][region_data['years'] == base_year].index[0]
    X_time = sm.add_constant(region_data['years'] - region_data['years'].min())
    
    # Calculate residuals
    predicted_yield = model.predict(X_time)
    residuals = region_data['yield'] - predicted_yield
    
    # Adjust yields to 2023 level
    base_predicted = predicted_yield[base_year_idx]
    adjusted_yield = base_predicted + residuals
    
    print(f"\n{region_name} - Adjusted Wheat Yields:")
    for year, adj_yield in zip(region_data['years'], adjusted_yield):
        print(f"Year {year}: {adj_yield:.2f} kg/ha")
    
    return adjusted_yield

# Section 4.6: Selection of Weather and Agricultural Input Index Design
def select_index(region_data, region_name, adjusted_yield):
    # Calculate correlations with all variables
    corr_rainfall, _ = pearsonr(adjusted_yield, region_data['rainfall'])
    corr_temperature, _ = pearsonr(adjusted_yield, region_data['temperature'])
    corr_npk, _ = pearsonr(adjusted_yield, region_data['npk'])
    corr_irrigated_area, _ = pearsonr(adjusted_yield, region_data['irrigated_area'])
    
    print(f"\n{region_name} - Correlation with Indices:")
    print(f"Rainfall: {corr_rainfall:.2f}")
    print(f"Temperature: {corr_temperature:.2f}")
    print(f"Amount of NPK Used: {corr_npk:.2f}")
    print(f"Net Irrigated Area: {corr_irrigated_area:.2f}")
    
    # Select the index with the highest absolute correlation
    correlations = {
        'Rainfall': abs(corr_rainfall),
        'Temperature': abs(corr_temperature),
        'Amount of NPK Used': abs(corr_npk),
        'Net Irrigated Area': abs(corr_irrigated_area)
    }
    selected_index = max(correlations, key=correlations.get)
    print(f"\n{region_name} - Selected Index (based on highest correlation): {selected_index}")
    
    # Perform regression with the selected index
    column_mapping = {
        'Rainfall': 'Rain Fall (in millimeters)',
        'Temperature': 'Temperature',
        'Amount of NPK Used': 'Amount of NPK Used',
        'Net Irrigated Area': 'Net irrigated Area (in Hectare)'
    }
    X_index = sm.add_constant(region_data['data'][column_mapping[selected_index]])
    index_model = sm.OLS(adjusted_yield, X_index).fit()
    
    # Perform robust regressions
    huber_index_model = sm.RLM(adjusted_yield, X_index, M=sm.robust.norms.HuberT()).fit()
    bisquare_index_model = sm.RLM(adjusted_yield, X_index, M=sm.robust.norms.TukeyBiweight()).fit()
    
    # Calculate forecasting errors for index regressions
    index_pred = index_model.predict(X_index)
    huber_index_pred = huber_index_model.predict(X_index)
    bisquare_index_pred = bisquare_index_model.predict(X_index)
    
    index_errors = {
        'OLS': {
            'MAE': mean_absolute_error(adjusted_yield, index_pred),
            'MAPE': mean_absolute_percentage_error(adjusted_yield, index_pred)
        },
        'Huber': {
            'MAE': mean_absolute_error(adjusted_yield, huber_index_pred),
            'MAPE': mean_absolute_percentage_error(adjusted_yield, huber_index_pred)
        },
        'Bisquare': {
            'MAE': mean_absolute_error(adjusted_yield, bisquare_index_pred),
            'MAPE': mean_absolute_percentage_error(adjusted_yield, bisquare_index_pred)
        }
    }
    
    print(f"\n{region_name} - Regression with {selected_index} - Forecasting Errors:")
    for model, metrics in index_errors.items():
        print(f"{model} - MAE: {metrics['MAE']:.2f}, MAPE: {metrics['MAPE']:.2%}")
    
    # Select the best index regression model
    best_index_model_name = min(index_errors, key=lambda k: index_errors[k]['MAE'])
    best_index_model = {
        'OLS': index_model,
        'Huber': huber_index_model,
        'Bisquare': bisquare_index_model
    }[best_index_model_name]
    print(f"\n{region_name} - Best Model for {selected_index} Regression: {best_index_model_name}")
    print(best_index_model.summary())
    
    # Perform regression with specified indices (NPK for West Bengal and Bihar, Rainfall for Uttar Pradesh)
    specified_indices = {
        'West Bengal': 'Amount of NPK Used',
        'Uttar Pradesh': 'Rain Fall (in millimeters)',
        'Bihar': 'Amount of NPK Used'
    }
    
    if region_name in specified_indices:
        specified_index = specified_indices[region_name]
        print(f"\n{region_name} - Regression with Specified Index ({specified_index}):")
        
        X_specified = sm.add_constant(region_data['data'][specified_index])
        
        # Perform OLS and robust regressions
        specified_model = sm.OLS(adjusted_yield, X_specified).fit()
        huber_specified_model = sm.RLM(adjusted_yield, X_specified, M=sm.robust.norms.HuberT()).fit()
        bisquare_specified_model = sm.RLM(adjusted_yield, X_specified, M=sm.robust.norms.TukeyBiweight()).fit()
        
        # Calculate forecasting errors
        specified_pred = specified_model.predict(X_specified)
        huber_specified_pred = huber_specified_model.predict(X_specified)
        bisquare_specified_pred = bisquare_specified_model.predict(X_specified)
        
        specified_errors = {
            'OLS': {
                'MAE': mean_absolute_error(adjusted_yield, specified_pred),
                'MAPE': mean_absolute_percentage_error(adjusted_yield, specified_pred)
            },
            'Huber': {
                'MAE': mean_absolute_error(adjusted_yield, huber_specified_pred),
                'MAPE': mean_absolute_percentage_error(adjusted_yield, huber_specified_pred)
            },
            'Bisquare': {
                'MAE': mean_absolute_error(adjusted_yield, bisquare_specified_pred),
                'MAPE': mean_absolute_percentage_error(adjusted_yield, bisquare_specified_pred)
            }
        }
        
        print(f"\n{region_name} - Regression with {specified_index} - Forecasting Errors:")
        for model, metrics in specified_errors.items():
            print(f"{model} - MAE: {metrics['MAE']:.2f}, MAPE: {metrics['MAPE']:.2%}")
        
        # Select the best model for specified index regression
        best_specified_model_name = min(specified_errors, key=lambda k: specified_errors[k]['MAE'])
        best_specified_model = {
            'OLS': specified_model,
            'Huber': huber_specified_model,
            'Bisquare': bisquare_specified_model
        }[best_specified_model_name]
        print(f"\n{region_name} - Best Model for {specified_index} Regression: {best_specified_model_name}")
        print(best_specified_model.summary())
        
        return selected_index, best_index_model, specified_index, best_specified_model
    else:
        return selected_index, best_index_model

# Execute the analysis for both regions
if __name__ == "__main__":
    for region_name, region_data in regions.items():
        print(f"\n=== Analysis for {region_name} ===")
        
        # 4.1
        calculate_time_trend_correlation(region_data, region_name)
        
        # 4.2 and 4.3
        ols_model = perform_ols_regression(region_data, region_name)
        
        # 4.4
        ols_model, huber_model, bisquare_model = perform_robust_regression(region_data, region_name, ols_model)
        
        # Select the model with the lowest MAE
        X_time = sm.add_constant(region_data['years'] - region_data['years'].min())
        errors = {
            'OLS': mean_absolute_error(region_data['yield'], ols_model.predict(X_time)),
            'Huber': mean_absolute_error(region_data['yield'], huber_model.predict(X_time)),
            'Bisquare': mean_absolute_error(region_data['yield'], bisquare_model.predict(X_time))
        }
        best_model_name = min(errors, key=errors.get)
        best_model = {'OLS': ols_model, 'Huber': huber_model, 'Bisquare': bisquare_model}[best_model_name]
        print(f"\n{region_name} - Best Model for Detrending: {best_model_name}")
        
        # 4.5
        adjusted_yield = adjust_wheat_yield(region_data, region_name, best_model)
        
        # 4.6
        result = select_index(region_data, region_name, adjusted_yield)
        if len(result) == 4:
            selected_index, best_index_model, specified_index, best_specified_model  = result
        else:
            selected_index, best_index_model = result


=== Analysis for West Bengal ===
West Bengal - Correlation between wheat yield and time trend: 0.88

West Bengal - OLS Regression Summary:
                                 OLS Regression Results                                
Dep. Variable:     Yield of Wheat (kg/Hectare)   R-squared:                       0.772
Model:                                     OLS   Adj. R-squared:                  0.759
Method:                          Least Squares   F-statistic:                     57.68
Date:                         Wed, 17 Sep 2025   Prob (F-statistic):           7.35e-07
Time:                                 22:22:26   Log-Likelihood:                -118.46
No. Observations:                           19   AIC:                             240.9
Df Residuals:                               17   BIC:                             242.8
Df Model:                                    1                                         
Covariance Type:                     nonrobust                      